# **Math Score model**

## Objectives

- Train and evaluate a model for predicting a student's math score

## Inputs

- outputs/datasets/filtered/math-score-dataset.csv
- outputs/datasets/split/math/math-test-score.csv
- outputs/datasets/split/math/math-train-score.csv
- outputs/datasets/split/math/math-test-vars.csv
- outputs/datasets/split/math/math-train-vars.csv

## Outputs

- A pipeline to predict a student's math score

## Additional comments

# Change working directory

Since this notebook exists in the jupyter_notebooks directory, we need to change the current working directory from the jupyter_notebooks directory to the workspace, so that any directories created in further codes cells are added in the correct place. 

We access the current directory with the OS packages' `getcwd()` method

In [ ]:
import os
current_directory = os.getcwd()
current_directory


We now want to set the working directory as the parent of the current working directory, jupyter_notebooks

- The `os.path.dirname()` method gets the parent directory
- The `os.chir()` method defines the new current directory
- We do this to access all of the project's files and directories, rather than those in the jupyter_notebooks directory

In [ ]:
os.chdir(os.path.dirname(current_directory))
print("You set a new current directory")

To make certain of things, we now use a code cell to confirm that we have set the current working directory properly

In [ ]:
current_directory = os.getcwd()
current_directory

Import packages:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.pipeline import Pipeline

### Feature Engineering
from feature_engine.encoding import OrdinalEncoder

### Feature Scaling
from sklearn.preprocessing import StandardScaler

### Feature  Selection
from sklearn.feature_selection import SelectFromModel

### ML algorithms 
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

## Math Score pipeline

Considering that the math_score variable is a continuous numerical value, our objective in this notebook is to construct a pipeline for a regression model. We have previously noted only weak correlation levels between the categorical feature variables and the numerical target variables, so it is entirely possible that a regression model will fail to perform well enough to fulfil the business requirements. If that is the case, we will switch to using a classification model, using bins. 

First of all, let's load the required datasets:

In [ ]:
df_maths = pd.read_csv('outputs/datasets/filtered/math-score-dataset.csv')
df_maths

Now we load the train and test sets. We use the `squeeze()` method to transform the dataframe into a Series, as this is what the math_train_score and math_test_score variables were initially produced as.

In [ ]:
math_train_vars = pd.read_csv('outputs/datasets/split/math/math-train-vars.csv')
math_train_score = pd.read_csv('outputs/datasets/split/math/math-train-score.csv').squeeze()
math_test_vars = pd.read_csv('outputs/datasets/split/math/math-test-vars.csv')
math_test_score = pd.read_csv('outputs/datasets/split/math/math-test-score.csv').squeeze()

## Pipeline

We will now define the pipeline that we will use. We know that our dataset's feature variables are categorical, so we need an ordinal categorical encoder step first. As we saw in the 07-FeatureEngineering notebook, none of the variables are sufficiently correlated to warrant a smart correlated selection step. We will then use a standard feature scaling and feature selection steps, and finally have the model step.

In [ ]:
def PipelineOptimization(model):
  pipeline_base = Pipeline([

      ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary', 
                                                  variables = ['gender', 'ethnicity', 'parental_education', 'lunch_program', 'test_preparation_course'] ) ),
       

      ("feature_scaling", StandardScaler() ),

      ("feature_selection",  SelectFromModel(model) ),

      ("model", model ),
       
    ])

  return pipeline_base

## Algorithm and Hyperparameter optimisation

We now need to determine the best algorithm to use, and then for that algorithm, the best hyperparameters. To do this, we will use the custom class defined in the [6 - Modeling and Evaluation - predict Tenure notebook of the Churnometer walkthrough project.](https://github.com/AdamBoley/churnometer/blob/main/jupyter_notebooks/06%20-%20Modeling%20and%20Evaluation%20-%20Predict%20Tenure.ipynb)



In [ ]:
from sklearn.model_selection import GridSearchCV
class HyperparameterOptimizationSearch:

    def __init__(self, models, parameters):
        self.models = models
        self.parameters = parameters
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, x, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model =  PipelineOptimization(self.models[key])

            parameters = self.parameters[key]
            grid_search = GridSearchCV(model, parameters, cv=cv, n_jobs=n_jobs, verbose=verbose, scoring=scoring)
            grid_search.fit(x, y)
            self.grid_searches[key] = grid_search

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, parameters):
            summary = {
                 'estimator': key,
                 'minimum_score': min(scores),
                 'maximum_score': max(scores),
                 'mean_score': np.mean(scores),
                 'standard_deviation_score': np.std(scores),
            }
            return pd.Series({**parameters,**summary})

        rows = []
        for k in self.grid_searches:
            parameters = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                result = self.grid_searches[k].cv_results_[key]        
                scores.append(result.reshape(len(parameters), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(parameters, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'minimum_score', 'mean_score', 'maximum_score', 'standard_deviation_score']
        columns = columns + [column for column in df.columns if column not in columns]

        return df[columns], self.grid_searches


We can now select the algorithms to compare:

In [ ]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

parameters_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

Now we call the custom function using the training data

In [ ]:
model_search = HyperparameterOptimizationSearch(models=models_quick_search, parameters=parameters_quick_search)
model_search.fit(math_train_vars, math_train_score, scoring='r2', n_jobs=-1, cv=5)

Now we can check which algorithm performed best

In [ ]:
model_grid_search_summary, model_grid_search_pipelines = model_search.score_summary(sort_by='mean_score')
model_grid_search_summary

As we can see from the grid_search_summary dataframe, none of the algorithms perform particularly well, but the LinearRegression model performs best. Per the [SKlearn LinearRegression page](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html?highlight=linearregression#sklearn.linear_model.LinearRegression), this algorithm has no hyperparameters that we can use to optimise performance.

However, the RandomForestRegressor performed almost as well - it has a higher maximum score, but the mean score is brought down by the lower minimum score. The RandomForestRegressor has 5 hyperparameters that we can tweak, so we should see if these can boost performance to acceptable levels

In [ ]:
models_search = {
    "RandomForestRegressor":RandomForestRegressor(random_state=0),
}

# Hyperparamters:
# https://docs.google.com/document/d/1PSug1nGF24NQHBByUzVFiNNKwvCMPLCc5x-ODfla28E/edit#

parameters_search = {
    "RandomForestRegressor":{'model__n_estimators': [100, 50, 140],
                             'model__max_depth': [None, 4, 15],
                             'model__min_samples_split': [2, 50],
                             'model__min_samples_leaf': [1, 50],
                             'model__max_leaf_nodes': [None, 50],
                            }
}


In [ ]:
param_search = HyperparameterOptimizationSearch(models=models_search, parameters=parameters_search)
param_search.fit(math_train_vars, math_train_score, scoring = 'r2', n_jobs=-1, cv=5)

In [ ]:
param_grid_search_summary, param_grid_search_pipelines = param_search.score_summary(sort_by='mean_score')
param_grid_search_summary

As we suspected, it looks like optimising the hyperparameters has failed to produce a model of adequate performance. Let's check the best performing models:

In [ ]:
best_model = model_grid_search_summary.iloc[0,0]
best_model

In [ ]:
best_model_and_params = param_grid_search_summary.iloc[0,0]
best_model_and_params

And then the parameters for those models:

In [ ]:
model_grid_search_pipelines[best_model].best_params_

In [ ]:
param_grid_search_pipelines[best_model_and_params].best_params_

And then the best pipeline:

In [ ]:
best_regressor_pipeline_1 = model_grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline_1

In [ ]:
best_regressor_pipeline_2 = param_grid_search_pipelines[best_model_and_params].best_estimator_
best_regressor_pipeline_2

We can now assess which feature variables are the most important. Note that we can only do this for the RandomForestRegressor algorithm (best_regressor_pipeline_2), since the LinearRegression algorithm (best_regressor_pipeline_1) has no feature_importances_ method. Running the code cell below with the LinearRegression pipeline throws and error.

In [ ]:
# after data cleaning and feat engine, the feature may space changes
data_cleaning_feat_eng_steps = 1 # how many data cleaning and feature engineering does your pipeline have?
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline_2.steps[:data_cleaning_feat_eng_steps])
                                        .transform(math_train_vars)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline_2['feature_selection'].get_support()].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
          'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline_2['feature_selection'].get_support()],
          'Importance': best_regressor_pipeline_2['model'].feature_importances_})
  .sort_values(by='Importance', ascending=False)
  )

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

This appears to support the conclusions we drew in the data analysis notebooks - the lunch_program variable has considerable importance, as to ethnicity and parental_education.

We can also evaluate the performance of our models on the train and test sets

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error 
import numpy as np

def regression_performance(math_train_vars, math_train_score, math_test_vars, math_test_score, pipeline):
	print("Model Evaluation \n")
	print("* Train Set")
	regression_evaluation(math_train_vars, math_train_score, pipeline)
	print("* Test Set")
	regression_evaluation(math_test_vars, math_test_score, pipeline)

def regression_evaluation(X, y, pipeline):
  prediction = pipeline.predict(X)
  print('R2 Score:', r2_score(y, prediction).round(3))  
  print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))  
  print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))  
  print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, prediction)).round(3))
  print("\n")


def regression_evaluation_plots(math_train_vars, math_train_score, math_test_vars, math_test_score, pipeline, alpha_scatter=0.5):
  pred_train = pipeline.predict(math_train_vars)
  pred_test = pipeline.predict(math_test_vars)


  fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
  sns.scatterplot(x=math_train_score , y=pred_train, alpha=alpha_scatter, ax=axes[0])
  sns.lineplot(x=math_train_score , y=math_train_score, color='red', ax=axes[0])
  axes[0].set_xlabel("Actual")
  axes[0].set_ylabel("Predictions")
  axes[0].set_title("Train Set")

  sns.scatterplot(x=math_test_score , y=pred_test, alpha=alpha_scatter, ax=axes[1])
  sns.lineplot(x=math_test_score , y=math_test_score, color='red', ax=axes[1])
  axes[1].set_xlabel("Actual")
  axes[1].set_ylabel("Predictions")
  axes[1].set_title("Test Set")

  plt.show()

Now we call the functions for the Linear Regression algorithm:

In [ ]:
regression_performance(math_train_vars, math_train_score, math_test_vars, math_test_score, best_regressor_pipeline_1)
regression_evaluation_plots(math_train_vars, math_train_score, math_test_vars, math_test_score, best_regressor_pipeline_1)

And the function call for the RandomForestRegressor:

In [ ]:
regression_performance(math_train_vars, math_train_score, math_test_vars, math_test_score, best_regressor_pipeline_2)
regression_evaluation_plots(math_train_vars, math_train_score, math_test_vars, math_test_score, best_regressor_pipeline_2)

The RandomForestRegressor appears to offer somewhat improved performance, but the R2 scores for both algorithms are poor.

As in the Churnometer project, we could try a Principal Component Analysis. However, we only have 5 feature variables, so this is unlikely to sufficiently improve performance. Instead, we will move straight to converting this task from regression to classification. To clearly separate matters, we will explore this classification task in the next notebook - 08b-math-score-model-classification.